In [1]:
import sympy as sp
import numpy as np

In [2]:
def generate_data(epsilon: float, precision) -> tuple[np.ndarray, np.ndarray]:
    return (
        np.array([[1, 1,],
                  [1, 1 + epsilon],
                  [1, 1 + epsilon]],
                 dtype=precision),
        np.array([2, epsilon, 4 + epsilon], dtype=precision))

First let's find the exact solution:

In [3]:
epsilon = sp.Symbol('\\varepsilon')
A_sym = sp.Matrix([[1, 1],
                   [1, 1 + epsilon],  # type: ignore
                   [1, 1 + epsilon]]) # type: ignore
b_sym = sp.Matrix([[2], [epsilon], [4 + epsilon]]) # type: ignore

x = (A_sym.transpose() @ A_sym).inv() @ A_sym.transpose() @ b_sym
x.simplify()
x

Matrix([
[1],
[1]])

Using simply an inverse to solve the problem does not yield great results (it just explodes with the 32-bit precision).

In [4]:
# Am I just going to write a naive implementation? OK
A32, b32 = generate_data(0.00001, np.float32)
x32 = np.linalg.inv((A32.transpose() @ A32)) @ A32.transpose() @ b32
A64, b64 = generate_data(0.0001, np.float64)
x64 = np.linalg.inv((A64.transpose() @ A64)) @ A64.transpose() @ b64
print(f'32-bit precision:\n{x32}\n\n64-bit precision:\n{x64}')

32-bit precision:
[1024.5    2047.5078]

64-bit precision:
[0.99999983 1.00000011]


In [15]:
def lstsq_svd(matrix: np.ndarray, column: np.ndarray) -> np.ndarray:
    U, S, Vh = np.linalg.svd(matrix)
    rank = S.shape[0]
    return Vh.transpose() @ ((U.transpose() @ column)[:rank] / S)

Using SVD seems stable enough to produce quite close results event for 32-bit numbers.

In [30]:
x32svd = lstsq_svd(A32, b32)
x64svd = lstsq_svd(A64, b64)
print(f'32-bit precision:\n{x32svd}\nresiduals:\n{x32svd - np.array([1, 1.])}\n')
print(f'64-bit precision:\n{x64svd}\nresiduals:\n{x64svd - np.array([1, 1.])}')

32-bit precision:
[0.9999966 1.0000032]
residuals:
[-3.39746475e-06  3.21865082e-06]

64-bit precision:
[1. 1.]
residuals:
[ 3.26383365e-12 -3.26405569e-12]


`lstsq` results are similar to the ones calculated using SVD. I assume the function uses either SVD ot other decomposition algorithm, probably QR decomposition.

In [31]:
x32np = np.linalg.lstsq(A32, b32, rcond=None)[0]
x64np = np.linalg.lstsq(A64, b64, rcond=None)[0]
print(f'32-bit precision:\n{x32np}\nresiduals:\n{x32np - np.array([1, 1.])}\n')
print(f'64-bit precision:\n{x64np}\nresiduals:\n{x64np - np.array([1, 1.])}')

32-bit precision:
[1.0006781 0.9993219]
residuals:
[ 0.00067806 -0.00067812]

64-bit precision:
[1. 1.]
residuals:
[ 8.96194230e-12 -8.96105412e-12]
